In [25]:
#install required libraries
!pip install transformers accelerate safetensors invisible-watermark>=2.0 diffusers

In [26]:
#Enter your token
YOUR_HF_TOKEN = ""


In order to use the SDXL model, you must first open a Hugging Face account, then click this link to accept the terms of use and fill in some information.
https://huggingface.co/stabilityai/stable-diffusion-xl-base-0.9

You need tokens to use this model. To do this, you can create a token by clicking this link. After clicking the New token button, enter a name, then select write in the role section, then click Generate a token. After these transactions, tokens will be created. You must have a Hugging Face account to be able to do this.
https://huggingface.co/settings/tokens

In [27]:
#import required library
from diffusers import DiffusionPipeline
import torch
import gc

In [28]:
#load sdxl-base-0.9 model
model_base = "stabilityai/stable-diffusion-xl-base-0.9"

pipe = DiffusionPipeline.from_pretrained(
    model_base,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    use_auth_token=YOUR_HF_TOKEN,
    )

#If you are seeing out-of-memory errors use this.
pipe.enable_model_cpu_offload()

In [29]:
#load sdxl-refiner-0.9 model
model_refiner = "stabilityai/stable-diffusion-xl-refiner-0.9"

pipe_refiner = DiffusionPipeline.from_pretrained(
    model_refiner,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    use_auth_token=YOUR_HF_TOKEN,
    )

pipe_refiner.enable_model_cpu_offload()

In [ ]:
#Enter the parameters. The parameters are entirely up to you. By adjusting the parameter values, you can increase or decrease the quality of the image.
def generate_images(prompt,negative_prompt):
    image_base = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        #height=768,
        #width=768,
        guidance_scale=9.0,
        num_inference_steps=70,
        eta = 0.2,
    ).images[0]

    image_refiner = pipe_refiner(
        prompt=prompt,
        negative_prompt = negative_prompt,
        #height=768,
        #width=768,
        image=image_base,
        num_inference_steps=70,
        strength=0.5,
        eta = 0.2,
    ).images[0]

    gc.collect()
    torch.cuda.empty_cache()

    return image_base, image_refiner

#Enter your prompt and negative_prompt
def print_image():

  prompt = "(colorful bird posing with extreme glowing colors:1. 8), (Sigma 85mm f/1. 4:0. 7), intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by krenz cushart and artem demura and alphonse mucha"
  negative_prompt = "incorrect proportions, ugly background, human, boring, three feet, crooked beak, out of frame, cartoon, sketch, painting, crossed eyes, (worst quality:2), (low quality:2), (normal quality:2), lowres, bad anatomy, normal quality, ((monochrome)), two beaks, words, signature, text, over exposure,two face , two head, two bird"

  base_image, refined_image = generate_images(prompt, negative_prompt)

  #Save images
  base_image.save("base_imag.jpeg")
  refined_image.save("refined_image.jpeg")

print_image()



https://huggingface.co/docs/diffusers/api/pipelines/stable_diffusion/stable_diffusion_xl